In [ ]:
def clean_text(text):
    stop_words = set(stopwords.words('english')) # obtain the stop words
    good_words = [] # save the correct words to consider like tokens
    tokenizer = RegexpTokenizer("[\w']+") # function to recognize the tokens
    words = tokenizer.tokenize(text) # tokenize the text 
    for word in words:
        # check if the word is lower and it isn't a stop word or a number
        if word.lower() not in stop_words and word.isalpha(): 
            word = PorterStemmer().stem(word) # use the stemmer function
            good_words.append(word.lower()) # insert the good token to lower case
        
    return good_words

In [ ]:
def create_vocabulary(df):
    ### Input == I use like input the dataset obtain in exercise 1 where i apply the clean text function
    ### Output == I obtain a vocabulary, the keys are all tokens (with no repeat) contained in the plot for the each rows
    ### for each token I define the index of the rows where the token is in the plot
    vocabulary = {}
    for i, row in df.iterrows():
            if len(df.at[i, "Plot"]) > 0:  # check if the list is empty or not to avoid the eventually error
                for word in df.at[i, "Plot"]: # bring the token from the list
                    if word in vocabulary.keys(): # insert the token into the vocabulary with the documents where this is present
                        if i not in vocabulary[word]:
                            vocabulary[word].append(i)
                    else:
                        vocabulary[word] = [i]
    return vocabulary

In [ ]:
def get_scraping(input_path):
    path = str("./"+input_path)

    filenames = os.listdir(path)
    for i in range(1, 301):
        filenames = os.listdir(path + '/' + str(i))

        for file in filenames:
            with open(path + '/' + str(i) + './article_'+str(file.split("_")[1].replace(".html", ""))+'.tsv', 'w', encoding="utf-8", newline='') as out_file: # create for each html article its article_i.tsv according to the professor requests!
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 
                                'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published',
                                'Characters', 'Setting', 'Url'])
                scrap_book(tsv_writer, path + '/' + str(i) + "/" + file)

# creating the final dataset
def finaldataset_tsv(initial_path):
    path = str("./"+initial_path)
    suffix = ".tsv" 
    filenames = os.listdir(path) #
    data2 = pd.DataFrame()

    for i in range(1, 301):
        filenames = os.listdir(path + '/' + str(i))
        for file in filenames:
            if file.endswith(suffix): 
                with open(path + '/' + str(i) + '/article_'+str(file.split("_")[1]), 'r', encoding="utf-8", newline='') as out_file:
                        df = pd.read_csv(out_file,sep = "\t")
                        if  df.loc[0,"Plot"] != " " and df.loc[0,"bookTitle"] != " ":
                            data2 = pd.concat([data2,df])
                            
    with open("finaloutput.tsv", "w", encoding="utf-8", newline="") as text_file: text_file.write(data2.to_csv(index=False)) 